# What is `madlibs`?

`madlibs` is a library that lets us create templates containing variables, and populate the variables with fillers. For example, given a template like `The {{person}} bought an {{object}}.` and values for `{{person}}` and `{{object}}` defined to be {`man`, `woman`} and {`car`, `spaceship`} respectively, we will generate four sentences:

    The woman bought a car.
    The man bought a car.
    The woman bought a spaceship.
    The man bought a spaceship.


## Using `madlibs`: Simple Example

Let us look at an example of how to use the library. Let us first import `MadLibs` and create a template.

In [1]:
from madlibs.madlibs import MadLibs

In [2]:
templates = {
    "s1": "The {{profession}} bought {{object}}.",
    "s2": "The {{person}} bought {{object}}.",
    "s3": "{{pronoun | title}} bought {{object}}.",
    "s4": "{{name}} the {{profession}} bought {{object}}."
}


We have created a collection of templates named `s1`, `s2`, `s3` and `s4`. Each template contains a some variables specified with curly brackets `{{...}}`. In this collection of four templates, we have the following variables:

1. `profession`
2. `object`
3. `person`
4. `pronoun`
5. `name`

Two observations are worth mentioning here. First, variables can be used multiple times in the same template collection, and will all get populated with the same value, as we will see. Second, we have embellished the `pronoun` variable with the operator `title`. This is a `jinja` filter; the jinja website has a [list of supported filters](https://jinja.palletsprojects.com/en/3.0.x/templates/#list-of-builtin-filters).


Next, we need to specify what values these variables can take. Note that the variables `person`, `name` and `pronoun` are not independent here. To allow for this, we will specify variables using a dictionary, where the fillers can be either a list of strings, or a list of dictionaries that defines groups like `person`, `name` and `pronoun`.

In [3]:
fillers = {
    "profession": ["doctor", "nurse", "janitor", "professor", "architect", "politician"],
    "object": ["a bagel", "an apple", "a car", "a bus"],
    "people": [
        {"name": "Jack", "person": "man", "pronoun": "he"}, 
        {"name": "Jill", "person": "woman", "pronoun": "she"}, 
        {"name": "John", "person": "grandfather", "pronoun": "he"}, 
        {"name": "Mary", "person": "grandmother", "pronoun": "she"}, 
    ],
    "animal": ["cat", "dog", "fish"]
}

Here, we see the the variables `profession` and `object` are keys in the dictionary. There is another key `people`, which does not show up in the templates, but it is mapped to a list whose elements contain `name`, `person` and `pronoun`. There is also a variable `animal` that is not used in any of the templates. As we will see, such unused variables get ignored. 

Let us now combine the templates and the fillers.

In [4]:
madlibs = MadLibs(templates=templates, fillers=fillers)

Using `madlibs`, we can generate groups of sentences by populating the templates with fillers. Calling `madlibs.generate()` produces an iterator consisting of pairs of variable assignments and the corresponding realizations of the templates.

Let us look at the first five. 

In [5]:
num_generated = 0

for assignments, sentences in madlibs.generate():

    s1 = sentences["s1"]
    s2 = sentences["s2"]
    s3 = sentences["s3"]
    s4 = sentences["s4"]
    
    print(assignments)
    print("\t", s1)
    print("\t", s2)
    print("\t", s3)
    print("\t", s4)
    
    num_generated += 1
    if num_generated >= 5:
        break 
    


{'name': 'Jack', 'pronoun': 'he', 'profession': 'doctor', 'object': 'a bagel', 'person': 'man'}
	 The doctor bought a bagel.
	 The man bought a bagel.
	 He bought a bagel.
	 Jack the doctor bought a bagel.
{'name': 'Jill', 'pronoun': 'she', 'profession': 'doctor', 'object': 'a bagel', 'person': 'woman'}
	 The doctor bought a bagel.
	 The woman bought a bagel.
	 She bought a bagel.
	 Jill the doctor bought a bagel.
{'name': 'John', 'pronoun': 'he', 'profession': 'doctor', 'object': 'a bagel', 'person': 'grandfather'}
	 The doctor bought a bagel.
	 The grandfather bought a bagel.
	 He bought a bagel.
	 John the doctor bought a bagel.
{'name': 'Mary', 'pronoun': 'she', 'profession': 'doctor', 'object': 'a bagel', 'person': 'grandmother'}
	 The doctor bought a bagel.
	 The grandmother bought a bagel.
	 She bought a bagel.
	 Mary the doctor bought a bagel.
{'name': 'Jack', 'pronoun': 'he', 'profession': 'doctor', 'object': 'an apple', 'person': 'man'}
	 The doctor bought an apple.
	 The man

Each `assignment` here is a dictionary, mapping variables to their instantiations. The keys in the dictionary `sentences` are based on the keys in the original `templates`. Note that the variable `fish` in the fillers is ignored.

## Numeric variables

Sometimes, we may want to define some variables to be numbers. We could, of course, specify a filler whose values are all numbers, but `madlibs` includes syntax to make this easier. Let us see an example. As before, let us define some templates first.

In [21]:
templates = {
    "0": "{{name}} bought {{number | range(2, 5)}} fruits.",
}

We have two variables here: `name` and `number`. But we have defined `number` as a range using the familiar python syntax for numeric ranges: `range(start, end, step)`. If `step` is not specified, as above, it is assumed to be `1`. We still need to specify the domain of `name`.

In [22]:
fillers = {
    "name": ["Jack", "Jill"]
}

Now, we can populate the template using the fillers.

In [23]:
madlibs = MadLibs(templates=templates, fillers=fillers)


for assignment, sentences in madlibs.generate():
    print(assignment)
    print(sentences)

{'name': 'Jack', 'number': '2'}
{'0': 'Jack bought 2 fruits.'}
{'name': 'Jack', 'number': '3'}
{'0': 'Jack bought 3 fruits.'}
{'name': 'Jack', 'number': '4'}
{'0': 'Jack bought 4 fruits.'}
{'name': 'Jill', 'number': '2'}
{'0': 'Jill bought 2 fruits.'}
{'name': 'Jill', 'number': '3'}
{'0': 'Jill bought 3 fruits.'}
{'name': 'Jill', 'number': '4'}
{'0': 'Jill bought 4 fruits.'}


We get the full cross product of names and numbers.

## Defining aliases for variables

We can define variables as belonging to certain types and share the variables. Let us see why we need to do this. Consider a MadLib with one template, defined below:

In [6]:
templates = {
    "0": "{{name}} bought a car.",
    "1": "{{name}} was not happy with the car."
}

There is only one variable here. Let us create a few fillers for `name`.

In [7]:
fillers = {
    "name": ["Alice", "Bob", "Charlie"]
}

We can create a madlib with the template and the fillers.

In [8]:
madlibs = MadLibs(templates=templates, fillers=fillers)

assignment, sentences = next(madlibs.generate())
print(assignment)
print(sentences)

{'name': 'Alice'}
{'0': 'Alice bought a car.', '1': 'Alice was not happy with the car.'}


We see that both instances of the `name` variable refer to the same filler. This seems reasonable. However, what if we want a template to be populated with two different names. For example,

In [9]:
templates = {
    "0": "{{name}} bought a car for {{possessive_pronoun}} friend {{name}}."
}

fillers = {
    "people": [
        {"name": "John", "possessive_pronoun": "his"},
        {"name": "Mary", "possessive_pronoun": "her"},        
    ]
}

madlibs = MadLibs(templates=templates, fillers=fillers)

for assignment, sentences in madlibs.generate():
    print(assignment)
    print(sentences)

{'name': 'John', 'possessive_pronoun': 'his'}
{'0': 'John bought a car for his friend John.'}
{'name': 'Mary', 'possessive_pronoun': 'her'}
{'0': 'Mary bought a car for her friend Mary.'}


Maybe this is not what we wanted. Let us try again, by defining variable names and restricting their types.

In [10]:
templates = {
    "0": '{{name1 | type("name")}} bought a car for {{possessive_pronoun}} friend {{name2 | type("name")}}.'
}

fillers = {
    "people": [
        {"name": "John", "possessive_pronoun": "his"},
        {"name": "Mary", "possessive_pronoun": "her"},        
    ]
}

madlibs = MadLibs(templates=templates, fillers=fillers)

items = madlibs.generate()

for assignment, sentences in madlibs.generate():
    print(assignment)
    print(sentences)

{'possessive_pronoun': 'his', 'name1': 'John', 'name2': 'John'}
{'0': 'John bought a car for his friend John.'}
{'possessive_pronoun': 'his', 'name1': 'John', 'name2': 'Mary'}
{'0': 'John bought a car for his friend Mary.'}
{'possessive_pronoun': 'her', 'name1': 'Mary', 'name2': 'John'}
{'0': 'Mary bought a car for her friend John.'}
{'possessive_pronoun': 'her', 'name1': 'Mary', 'name2': 'Mary'}
{'0': 'Mary bought a car for her friend Mary.'}


Here, we have specified type constraints using the syntax `type("name")`, where `name` is a known filler. Essentially we have defined `name1` and `name2` to be variables whose domain is specified by the filler `name`.  

This may be better, but we are still not quite there. We need a constraint that says that `name1` and `name2` should be different from each other.

In [11]:
templates = {
    "0": '{{name1 | type("name")}} bought a car for {{possessive_pronoun}} friend {{name2 | type("name") | not_equals("name1")}}.'
}

fillers = {
    "people": [
        {"name": "John", "possessive_pronoun": "his"},
        {"name": "Mary", "possessive_pronoun": "her"},        
    ]
}

madlibs = MadLibs(templates=templates, fillers=fillers)

items = madlibs.generate()

for assignment, sentences in madlibs.generate():
    print(assignment)
    print(sentences)

{'possessive_pronoun': 'his', 'name1': 'John', 'name2': 'Mary'}
{'0': 'John bought a car for his friend Mary.'}
{'possessive_pronoun': 'her', 'name1': 'Mary', 'name2': 'John'}
{'0': 'Mary bought a car for her friend John.'}


We have added an extra constraint for `name2` saying that it should not be equal to `name1`. This gives us what we need. There are four such constraints currently defined: `equals`, `not_equals`, `less_than` and `greater_than`. The vocabulary of these constraints will grow.

Let us look at another example, where the constraints are applied to numeric variables.

In [30]:
templates = {
    "s0": '{{name1 | type("name")}} bought {{number1 | range(2, 5)}} cars for {{possessive_pronoun}} friend.',
    "s1": '{{name2 | type("name") | not_equals("name1")}} now has {{number2 | range(2,5) | greater_than("number1")}} cars.'
}

fillers = {
    "people": [
        {"name": "John", "possessive_pronoun": "his"},
        {"name": "Mary", "possessive_pronoun": "her"},        
    ]
}

madlibs = MadLibs(templates=templates, fillers=fillers)

items = madlibs.generate()

for assignment, sentences in madlibs.generate():
    print(assignment)
    for sentence_key in sentences:
        print(f"\t{sentence_key}: {sentences[sentence_key]}")


{'possessive_pronoun': 'his', 'name1': 'John', 'number2': '3', 'name2': 'Mary', 'number1': '2'}
	s0: John bought 2 cars for his friend.
	s1: Mary now has 3 cars.
{'possessive_pronoun': 'his', 'name1': 'John', 'number2': '4', 'name2': 'Mary', 'number1': '2'}
	s0: John bought 2 cars for his friend.
	s1: Mary now has 4 cars.
{'possessive_pronoun': 'his', 'name1': 'John', 'number2': '4', 'name2': 'Mary', 'number1': '3'}
	s0: John bought 3 cars for his friend.
	s1: Mary now has 4 cars.
{'possessive_pronoun': 'her', 'name1': 'Mary', 'number2': '3', 'name2': 'John', 'number1': '2'}
	s0: Mary bought 2 cars for her friend.
	s1: John now has 3 cars.
{'possessive_pronoun': 'her', 'name1': 'Mary', 'number2': '4', 'name2': 'John', 'number1': '2'}
	s0: Mary bought 2 cars for her friend.
	s1: John now has 4 cars.
{'possessive_pronoun': 'her', 'name1': 'Mary', 'number2': '4', 'name2': 'John', 'number1': '3'}
	s0: Mary bought 3 cars for her friend.
	s1: John now has 4 cars.
